In [1]:
import os
import re
import ast
import typing
import requests
import dask.dataframe as dd
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, datetime
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

%matplotlib inline

np.random.seed(31415)

sns.set(rc={'figure.figsize':(15,3)})
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Loading Data

In [2]:
i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
#for year in [2023, 2022, 2021, 2020]:
    #for month, month_name in i2m:
        #os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        #os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        #os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

In [33]:
# Mount Google Drive (only required if running from Google Colab)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [34]:
df_2020_03 = pd.read_csv('/content/drive/MyDrive/Capstone_Project/data/2020_03_Marc_BicingNou_ESTACIONS.csv')

In [35]:
df_2020_03.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl
0,1,24,23,1,16,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25
1,2,5,4,1,19,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25
2,3,6,6,0,18,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25
3,4,4,3,1,15,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25
4,5,14,14,0,25,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25


In [36]:
df_2020_03.shape

(2408419, 13)

In [37]:
df_2020_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2408419 entries, 0 to 2408418
Data columns (total 13 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   station_id                            int64  
 1   num_bikes_available                   int64  
 2   num_bikes_available_types.mechanical  int64  
 3   num_bikes_available_types.ebike       int64  
 4   num_docks_available                   int64  
 5   is_installed                          int64  
 6   is_renting                            int64  
 7   is_returning                          int64  
 8   last_reported                         float64
 9   is_charging_station                   bool   
 10  status                                object 
 11  last_updated                          int64  
 12  ttl                                   int64  
dtypes: bool(1), float64(1), int64(10), object(1)
memory usage: 222.8+ MB


In [38]:
missing_val = df_2020_03.isna().sum()

In [39]:
print(missing_val)

station_id                              0
num_bikes_available                     0
num_bikes_available_types.mechanical    0
num_bikes_available_types.ebike         0
num_docks_available                     0
is_installed                            0
is_renting                              0
is_returning                            0
last_reported                           0
is_charging_station                     0
status                                  0
last_updated                            0
ttl                                     0
dtype: int64


## Add stations info

In [40]:
station_df = pd.read_csv('/content/drive/MyDrive/Capstone_Project/Informacio_Estacions_Bicing.csv')
station_df.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,is_charging_station,nearby_distance,_ride_code_support,rental_uris,cross_street
0,1,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",8013,45,True,1000.0,True,NaN,NaN
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17.0,"C/ ROGER DE FLOR, 126",8013,29,True,1000.0,True,NaN,NaN
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11.0,"C/ NÀPOLS, 82",8013,27,True,1000.0,True,NaN,NaN
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8.0,"C/ RIBES, 13",8013,21,True,1000.0,True,NaN,NaN
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018,39,True,1000.0,True,NaN,NaN


In [41]:
df_2020_03_info = pd.merge(df_2020_03, station_df, how='left') # TODO: Handle NaN station information

In [42]:
df_2020_03_info.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,nearby_distance,_ride_code_support,rental_uris,cross_street
0,1,24,23,1,16,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",8013.0,45.0,1000.0,True,NaN,NaN
1,2,5,4,1,19,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17.0,"C/ ROGER DE FLOR, 126",8013.0,29.0,1000.0,True,NaN,NaN
2,3,6,6,0,18,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11.0,"C/ NÀPOLS, 82",8013.0,27.0,1000.0,True,NaN,NaN
3,4,4,3,1,15,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8.0,"C/ RIBES, 13",8013.0,21.0,1000.0,True,NaN,NaN
4,5,14,14,0,25,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018.0,39.0,1000.0,True,NaN,NaN


In [43]:
df_2020_03_info.isna().sum()

station_id                                    0
num_bikes_available                           0
num_bikes_available_types.mechanical          0
num_bikes_available_types.ebike               0
num_docks_available                           0
is_installed                                  0
is_renting                                    0
is_returning                                  0
last_reported                                 0
is_charging_station                           0
status                                        0
last_updated                                  0
ttl                                           0
name                                      25409
physical_configuration                    25409
lat                                       25409
lon                                       25409
altitude                                  25409
address                                   25409
post_code                                 25409
capacity                                

We don't have the info data for all the stations - Shere is working on this. I will drop for now all the rows with the capacity missing.

In [44]:
df_2020_03_info_cleaned = df_2020_03_info.dropna(subset=['capacity'])

In [45]:
df_2020_03_info_cleaned.head(4)

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,nearby_distance,_ride_code_support,rental_uris,cross_street
0,1,24,23,1,16,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",8013.0,45.0,1000.0,True,NaN,NaN
1,2,5,4,1,19,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17.0,"C/ ROGER DE FLOR, 126",8013.0,29.0,1000.0,True,NaN,NaN
2,3,6,6,0,18,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11.0,"C/ NÀPOLS, 82",8013.0,27.0,1000.0,True,NaN,NaN
3,4,4,3,1,15,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8.0,"C/ RIBES, 13",8013.0,21.0,1000.0,True,NaN,NaN


In [46]:
df_2020_03_info_cleaned.isna().sum()

station_id                                    0
num_bikes_available                           0
num_bikes_available_types.mechanical          0
num_bikes_available_types.ebike               0
num_docks_available                           0
is_installed                                  0
is_renting                                    0
is_returning                                  0
last_reported                                 0
is_charging_station                           0
status                                        0
last_updated                                  0
ttl                                           0
name                                          0
physical_configuration                        0
lat                                           0
lon                                           0
altitude                                      0
address                                       0
post_code                                     0
capacity                                

In [47]:
hh = df_2020_03_info_cleaned[df_2020_03_info_cleaned.num_docks_available > df_2020_03_info_cleaned.capacity]

In [48]:
hh.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,nearby_distance,_ride_code_support,rental_uris,cross_street
29,32,0,0,0,23,0,0,0,1.573802e+09,True,PLANNED,1583017224,25,LA BARCELONETA (CN BARCELONETA),ELECTRICBIKESTATION,41.373691,2.188928,4.0,LA BARCELONETA (CN BARCELONETA),8003.0,15.0,1000.0,True,NaN,NaN
104,111,2,2,0,25,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,C/ ARAGÓN 147,ELECTRICBIKESTATION,41.384753,2.154675,21.0,C/ ARAGÓN 147,8015.0,19.0,1000.0,True,NaN,NaN
274,285,4,0,4,19,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ MALATS, 98-100",ELECTRICBIKESTATION,41.436671,2.186105,35.0,"C/ MALATS, 98-100",8030.0,15.0,1000.0,True,NaN,NaN
418,469,4,1,3,13,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ SANT FRANCESC XAVIER, 1",ELECTRICBIKESTATION,41.441700,2.178155,72.0,"C/ SANT FRANCESC XAVIER, 1",8016.0,12.0,1000.0,True,NaN,NaN
427,483,0,0,0,30,1,1,1,1.583017e+09,True,IN_SERVICE,1583017224,25,"C/ LISBOA, 1",ELECTRICBIKESTATION,41.429562,2.158968,79.0,"C/ LISBOA, 1",8032.0,29.0,1000.0,True,NaN,NaN


In [49]:
hh.shape

(32885, 25)

There are some cases where num_docks_available is greater than capacity, which makes no sense.

In [50]:
df_2020_03_info_cleaned.status.unique()

array(['IN_SERVICE', 'PLANNED', 'NOT_IN_SERVICE', 'MAINTENANCE'],
      dtype=object)

TODO: Select in the future just the values with "Status" -> "In Service" (then delete this variable)

In [51]:
def get_datetime(miliseconds: int):
  return datetime.fromtimestamp(miliseconds)

def get_var_date(df: pd.DataFrame):
  df['date'] = pd.to_datetime(df['last_reported'].apply(lambda x: get_datetime(x)))
  df['month'] = df['date'].dt.month
  df['year'] = df['date'].dt.year
  df['hour'] = df['date'].dt.hour
  df['day'] = df['date'].dt.day
  return df

In [52]:
def get_date(row):
  return datetime(int(row['year']), int(row['month']), int(row['day']), int(row['hour']))

In [53]:
def process_df(df: pd.DataFrame):
    # Drop duplicated rows
    df_2 = df.drop_duplicates()
    # Drop rows with NaN in the variable last_reported
    df_2 = df_2.dropna(subset=['last_reported'], axis=0)
    # Create date variables from timestamp
    df_2 = get_var_date(df_2)
    # Create the datetime variable
    df_2['date_time'] = df_2.apply(lambda row: get_date(row), axis=1)
    # Calculate the percentage of docks available
    df_2['percentage_docks_available'] = df_2['num_docks_available'] / df_2['capacity']
    # Sort the remaining data by last_reported
    df_2 = df_2.sort_values('last_reported', ascending=True)
    # Select only the necessary columns for now
    my_var = ['station_id', 'year', 'month', 'day', 'hour', 'percentage_docks_available']
    df_2 = df_2[my_var]
    # Group by station and date, and calculate the mean
    df_2 = df_2.groupby(['station_id', 'year', 'month', 'day', 'hour']).mean(numeric_only=True).reset_index()

    return df_2

In [83]:
new_df_2020_03_info = process_df(df_2020_03_info_cleaned)

In [84]:
new_df_2020_03_info.head()

,station_id,year,month,day,hour,percentage_docks_available
0,1,2020,2,29,22,0.355556
1,1,2020,2,29,23,0.288889
2,1,2020,3,1,0,0.140741
3,1,2020,3,1,1,0.101852
4,1,2020,3,1,2,0.048148


In [85]:
new_df_2020_03_info.shape

(163446, 6)

In [86]:
new_df_2020_03_info = new_df_2020_03_info.sort_values(by=['station_id', 'year', 'month', 'day', 'hour']).reset_index(drop=True)

In [87]:
new_df_2020_03_info.head()

,station_id,year,month,day,hour,percentage_docks_available
0,1,2020,2,29,22,0.355556
1,1,2020,2,29,23,0.288889
2,1,2020,3,1,0,0.140741
3,1,2020,3,1,1,0.101852
4,1,2020,3,1,2,0.048148


We can see some wrong years (1970 or 2019) generated using the timestamp. -> En la limpieza de datos inicial tener en cuenta esto para elminarlo. Ahora no lo elimino porque estoy viendo como hacer esto de las variables contexto y así no me paro.

In [88]:
# De momento me quedo en este dataset solo lo que tenga año 2020 y mes 3, que es el ejemplo que he cogido:
new_df_2020_03_info = new_df_2020_03_info[(new_df_2020_03_info['year'] == 2020) & (new_df_2020_03_info['month'] == 3)].reset_index(drop=True)

In [133]:
new_df_2020_03_info.head(30)

,station_id,year,month,day,hour,percentage_docks_available
0,1,2020,3,1,0,0.140741
1,1,2020,3,1,1,0.101852
2,1,2020,3,1,2,0.048148
3,1,2020,3,1,3,0.000000
4,1,2020,3,1,4,0.000000
5,1,2020,3,1,5,0.000000
6,1,2020,3,1,6,0.038889
7,1,2020,3,1,7,0.085185
8,1,2020,3,1,8,0.103704
9,1,2020,3,1,9,0.222222


In [130]:
def calculate_context_variables(df):
    # Número de desplazamientos
    max_shift = 4

    # Lista para almacenar las filas con las variables de contexto calculadas
    context_rows = []

    # Iterar sobre cada estación única
    for station_id in df['station_id'].unique():
        # Filtrar el DataFrame por la estación actual
        station_df = df[df['station_id'] == station_id].copy().reset_index(drop=True)

        # Iterar sobre cada fila del DataFrame de la estación actual
        for i in range(len(station_df)):
            # Verificar si la fila actual se ha utilizado para calcular el contexto
            if i % (max_shift + 1) != 0:
                continue

            current_row = station_df.iloc[i]
            context_values = []

            # Iterar sobre los desplazamientos para calcular las variables de contexto
            for shift in range(1, max_shift + 1):
                context_index = i - shift

                # Ajustar el índice si se sale del rango del DataFrame
                context_index %= len(station_df)

                context_value = station_df.iloc[context_index]['percentage_docks_available']
                context_values.append(context_value)

            # Agregar los datos de la fila actual y las variables de contexto a la lista
            context_row = list(current_row) + context_values
            context_rows.append(context_row)

    # Convertir la lista de filas con variables de contexto en un DataFrame
    df_ctx = pd.DataFrame(context_rows, columns=list(df.columns) + [f'ctx-{shift}' for shift in range(1, max_shift + 1)])

    return df_ctx

In [131]:
df_ctx = calculate_context_variables(new_df_2020_03_info)

In [132]:
df_ctx.head(10)

,station_id,year,month,day,hour,percentage_docks_available,ctx-1,ctx-2,ctx-3,ctx-4
0,1.0,2020.0,3.0,1.0,0.0,0.140741,1.022222,1.022222,0.720370,0.688889
1,1.0,2020.0,3.0,1.0,5.0,0.000000,0.000000,0.000000,0.048148,0.101852
2,1.0,2020.0,3.0,1.0,10.0,0.331481,0.222222,0.103704,0.085185,0.038889
3,1.0,2020.0,3.0,1.0,15.0,0.705556,0.650000,0.572222,0.581481,0.490741
4,1.0,2020.0,3.0,1.0,20.0,0.545455,0.453704,0.379630,0.444444,0.338889
5,1.0,2020.0,3.0,2.0,1.0,0.207407,0.253704,0.303704,0.390741,0.521368
6,1.0,2020.0,3.0,2.0,6.0,0.409259,0.262963,0.222222,0.222222,0.222222
7,1.0,2020.0,3.0,2.0,11.0,0.737037,0.807407,0.885185,0.838889,0.629630
8,1.0,2020.0,3.0,2.0,16.0,0.298990,0.470370,0.548148,0.666667,0.679630
9,1.0,2020.0,3.0,2.0,21.0,0.000000,0.010101,0.150000,0.129630,0.225641


In [102]:
for station_id in tqdm(new_df_2020_03_info.station_id.unique()):
    station_df = new_df_2020_03_info[new_df_2020_03_info.station_id == station_id]
    station_df['ctx-4'] = station_df.percentage_docks_available.shift(4)
    station_df['ctx-3'] = station_df.percentage_docks_available.shift(3)
    station_df['ctx-2'] = station_df.percentage_docks_available.shift(2)
    station_df['ctx-1'] = station_df.percentage_docks_available.shift(1)

    # Eliminar filas con NaN en las columnas de contexto para evitar data leakage
    station_df = station_df.dropna(subset=['ctx-4', 'ctx-3', 'ctx-2', 'ctx-1'])

    df_ctx = pd.concat([station_df, df_ctx], axis=0).reset_index()

100%|██████████| 436/436 [00:21<00:00, 20.66it/s]


In [103]:
df_ctx.head(40)

,station_id,year,month,day,hour,percentage_docks_available,ctx-4,ctx-3,ctx-2,ctx-1
0,516,2020,3,5,1,0.932540,0.976190,0.952381,0.952381,0.952381
1,516,2020,3,5,2,0.904762,0.952381,0.952381,0.952381,0.932540
2,516,2020,3,5,3,0.904762,0.952381,0.952381,0.932540,0.904762
3,516,2020,3,5,4,0.904762,0.952381,0.932540,0.904762,0.904762
4,516,2020,3,5,5,0.904762,0.932540,0.904762,0.904762,0.904762
5,516,2020,3,5,6,0.904762,0.904762,0.904762,0.904762,0.904762
6,516,2020,3,5,7,0.904762,0.904762,0.904762,0.904762,0.904762
7,516,2020,3,5,8,0.904762,0.904762,0.904762,0.904762,0.904762
8,516,2020,3,5,9,0.904762,0.904762,0.904762,0.904762,0.904762
9,516,2020,3,5,10,0.908730,0.904762,0.904762,0.904762,0.904762


In [104]:
df_ctx.shape

(482661, 10)

In [105]:
df_ctx.isna().sum()

station_id                    0
year                          0
month                         0
day                           0
hour                          0
percentage_docks_available    0
ctx-4                         0
ctx-3                         0
ctx-2                         0
ctx-1                         0
dtype: int64